In [1]:
import numpy as np
from time import time
from driver import Model

In [2]:
myip = Model("bitstream/cnn.bit")

In [3]:
myip.debug=False

In [4]:
# load test data, check accuracy against software implementation
data = np.int32(np.load("test_x.npy"))
gold = np.load("test_y.npy")

In [5]:
myip.setCNNWeights(np.load("CNN_weights.npy"))
myip.setCNNBias(np.load("CNN_bias.npy"))
myip.setDenseWeights(np.load("dense_weights.npy"))
myip.setDenseBias(np.load("dense_bias.npy"))

In [6]:
# sample data
data[0,0,:]

array([ -5427,  -4580,   2832,  -1489,  -6022, -11662], dtype=int32)

In [9]:
myip.set_threshold(0.98)
threshld_miss_count = 0
confusion_matrix = np.zeros((4,4))

start_time = time()
for i in range(0, data.shape[0]):
    
#     print("-"*40 + f"Data: {i}" + "-"*40)
    gold_result = np.argmax(gold[i])
    prediction_above_threshold = False
    
    for j in range(0, data.shape[1]):
        
        # run inference
        predicted = myip.inference(data[i,j,:])
        if predicted != -1:
            prediction_above_threshold = True
            break
        
    if prediction_above_threshold:
        confusion_matrix[gold_result,predicted] += 1
    else:
        print("not confident enough to give prediction")
        threshld_miss_count += 1
    myip.resetBuffer()

print(f"total time for {data.shape[0] * data.shape[1]} inferences = {time() - start_time}")
print(f"Average inference time = {(time() - start_time)/(data.shape[0] * data.shape[1])}")
print(f"Max inference frequency = {(data.shape[0] * data.shape[1])/(time() - start_time)}")
print(f"Miss rate = {threshld_miss_count/data.shape[0]*100}%")

print(confusion_matrix)

not confident enough to give prediction
not confident enough to give prediction
not confident enough to give prediction
not confident enough to give prediction
not confident enough to give prediction
not confident enough to give prediction
total time for 55200 inferences = 10.413133144378662
Average inference time = 0.0001886962159820225
Max inference frequency = 5299.237802027473
Miss rate = 0.8152173913043478%
[[162.   0.   0.   2.]
 [  1. 134.   3.   1.]
 [  0.  10. 226.   4.]
 [  0.   0.   1. 186.]]
